In [4]:
import polars as pl
import pandas as pd
import numpy as np

In [15]:
files = [f"/media/HDD01/mbck/dataSet/preprocessing/window_{i}00/info" for i in [6,9,12,15,18]]

for file in files:
    pl.from_pandas(pd.read_feather(file + ".ft")).write_ipc(file + ".arrow")

In [12]:
%timeit pd.read_feather(file+ ".ft")

515 ms ± 25.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit pl.read_ipc(file + ".arrow")

161 µs ± 5.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


checkout joining

In [2]:
pl_preprocessing_info = pl.read_ipc("/media/HDD01/mbck/dataSet/preprocessing/window_300/info.arrow")
pd_preprocessing_info = pd.read_feather("/media/HDD01/mbck/dataSet/preprocessing/window_300/info.ft")

pl_base_info = pl.read_ipc("/media/HDD01/mbck/dataSet/dataset_info/V7.arrow")
pd_base_info = pd.read_feather("/media/HDD01/mbck/dataSet/dataset_info/V7.arrow")

In [5]:
preloaded_data ={k:np.array(v) for k, v in pl.read_ipc("/media/HDD01/mbck/dataSet/raw/total_dataset.arrow").iter_rows()}

In [6]:
merged =  pl_preprocessing_info.join(pl_base_info, left_on="gid", right_on="gid", how="left")

In [7]:
import numpy as np

data_values = np.array(list(preloaded_data.values()), dtype=object)
data_gid_indexe = {k:i for i, k in enumerate(preloaded_data.keys())}

In [8]:
merged.with_columns([pl.col("gid").apply(lambda x: data_gid_indexe[x])])

/tmp/ipykernel_275712/2943937819.py:1: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  merged.with_columns([pl.col("gid").apply(lambda x: data_gid_indexe[x])])
/tmp/ipykernel_275712/2943937819.py:1: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("gid").map_elements(lambda x: ...)
with this one instead:
  + pl.col("gid").replace(data_gid_indexe)

  merged.with_columns([pl.col("gid").apply(lambda x: data_gid_indexe[x])])


gid,uuid,tags,label,relevant,split
i64,str,struct[1],str,bool,str
0,"""abcac881-acdd-…",{4500},"""test_mission""",true,"""train"""
0,"""cbae87b7-8ee4-…",{4501},"""test_mission""",true,"""train"""
0,"""78283b53-1744-…",{4502},"""test_mission""",true,"""train"""
0,"""2cc844ec-1f75-…",{4503},"""test_mission""",true,"""train"""
0,"""5eb68de8-752a-…",{4504},"""test_mission""",true,"""train"""
0,"""1c19a831-951f-…",{4505},"""test_mission""",true,"""train"""
0,"""8c263ff5-d3e5-…",{4506},"""test_mission""",true,"""train"""
0,"""a17d8d03-e851-…",{4507},"""test_mission""",true,"""train"""
0,"""dd4a3fe5-9531-…",{4508},"""test_mission""",true,"""train"""


In [25]:
%timeit data_values[5]

41.8 ns ± 0.331 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [61]:
%timeit pl_preprocessing_info.join(pl_base_info, left_on="gid", right_on="gid", how="left")

856 ms ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%timeit pd_preprocessing_info.merge(pd_base_info, left_on="gid", right_on="gid", how="left")

6.41 s ± 110 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [3]:
pl_df = pl_preprocessing_info.join(pl_base_info, left_on="gid", right_on="gid", how="left")

In [60]:
pd_df = pd_preprocessing_info.merge(pd_base_info, left_on="gid", right_on="gid", how="left")

acces time for one element

In [30]:
element = "2dc0c98c-8f4c-4e08-af00-220a6bb51217"

In [63]:
%timeit pl_df.filter(pl.any_horizontal(pl.col('uuid') == element))

49.8 ms ± 598 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
%timeit pd_df[pd_df["uuid"] == element]

1.77 s ± 5.07 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


iloc/select by index

In [28]:
from random import randint
rand_max = pl_df.shape[0]

In [105]:
pl_df[randint(0, rand_max)].to_numpy()

array([['dc15c7e8-536c-4bb6-8d65-e7983910e747',
        'aac9d83e-88a3-4fd0-b3a2-261bccdfc505', {'timestamp': 25414},
        'real_mission', True, 'validation']], dtype=object)

In [69]:
%timeit pl_df[randint(0, rand_max)]

2.67 µs ± 8.43 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [70]:
%timeit pd_df.iloc[randint(0, rand_max)]

24.8 µs ± 301 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


check filtering

In [72]:
base_filter = pl.col("label").is_in(["test_mission", "real_mission"])

In [74]:
%timeit pl_df.filter(pl.col("relevant") & base_filter)

386 ms ± 9.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
%timeit pd_df.query(" and ".join(["relevant", "label in ['test_mission', 'real_mission']"]))

4.33 s ± 10.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


group by

In [84]:
%timeit dict(tuple(pl_df.group_by("split")))

<magic-timeit>:1: DeprecationWarning: `group_by` iteration will change to always return group identifiers as tuples. Pass `by` as a list to silence this warning, e.g. `group_by(['split'])`.


1.59 s ± 59.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [83]:
%timeit dict(tuple(pd_df.groupby("split")))

5.74 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
